In [1]:
'''
    @Author: King
    @Date: 2019.05.16
    @Purpose: Spark2.1.0+入门：读写Parquet(DataFrame)(Python版)
    @Introduction:   Spark2.1.0+入门：读写Parquet(DataFrame)(Python版)
    @Datasets: 
    @Link : 
    @Reference : Spark2.1.0+入门：读写Parquet(DataFrame)(Python版)
'''

'\n    @Author: King\n    @Date: 2019.05.16\n    @Purpose: Spark2.1.0+入门：读写Parquet(DataFrame)(Python版)\n    @Introduction:   Spark2.1.0+入门：读写Parquet(DataFrame)(Python版)\n    @Datasets: \n    @Link : \n    @Reference : Spark2.1.0+入门：读写Parquet(DataFrame)(Python版)\n'

![作者](../img/bigdata-roadmap.jpg)
【版权声明】博客内容由厦门大学数据库实验室拥有版权，未经允许，请勿转载！


## 一、 Spark2.1.0+入门：读写Parquet(DataFrame)(Python版)

Spark SQL可以支持Parquet、JSON、Hive等数据源，并且可以通过JDBC连接外部数据源。前面的介绍中，我们已经涉及到了JSON、文本格式的加载，这里不再赘述。这里介绍Parquet，下一节会介绍JDBC数据库连接。

Parquet是一种流行的列式存储格式，可以高效地存储具有嵌套字段的记录。Parquet是语言无关的，而且不与任何一种数据处理框架绑定在一起，适配多种语言和组件，能够与Parquet配合的组件有：
* 查询引擎: Hive, Impala, Pig, Presto, Drill, Tajo, HAWQ, IBM Big SQL
* 计算框架: MapReduce, Spark, Cascading, Crunch, Scalding, Kite
* 数据模型: Avro, Thrift, Protocol Buffers, POJOs

#### (1)、 引入 pyspark 库

In [2]:
# 引入 pyspark 库
from pyspark.sql import SparkSession

#### (2)、SparkSession 导入数据

In [3]:
spark=SparkSession.builder.getOrCreate()
df = spark.read.json("../resources/people.json")
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



#### (3)、将DataFrame保存成parquet文件

即将 JSON 数据 写入到目录"../resources/newpeople.parquet"中

In [4]:
df.write.parquet("../resources/newpeople.parquet")

上面命令执行后，可以看到”../resources”这个目录下多了一个newpeople.parquet，不过，注意，这不是一个文件，而是一个目录（不要被newpeople.parquet中的圆点所迷惑，文件夹名称也可以包含圆点），也就是说，peopleDF.write.parquet(“../resources/newpeople.parquet”)括号里面的参数是文件夹，不是文件名。下面我们可以进入newpeople.parquet目录，会发现下面2个文件：
```
    part-r-00000-8d3a120f-b3b5-4582-b26b-f3693df80d45.snappy.parquet
    _SUCCESS
```
这2个文件都是刚才保存生成的。现在问题来了，如果我们要再次把这个刚生成的数据又加载到DataFrame中，应该加载哪个文件呢？很简单，只要加载newpeople.parquet目录即可，而不是加载这2个文件，语句如下：

In [8]:
parquetFileDF = spark.read.parquet("../resources/newpeople.parquet")

parquetFileDF.createOrReplaceTempView("parquetFile")

namesDF = spark.sql("SELECT * FROM parquetFile")

namesDF.rdd.foreach(lambda person: print(person.name))

print(namesDF.collect())



[Row(age=None, name='Michael'), Row(age=30, name='Andy'), Row(age=19, name='Justin')]
